In [1]:
from importlib import reload

In [2]:
import param

In [3]:
# note: not yet sure where code will end up
import param.serialize as serialize

## Basic example

In [4]:
%%writefile model.py

import param
class Thing(param.Parameterized):
    a = param.Number(default=2, bounds=(0,3))
    b = param.Boolean(default=False)
    c = param.List(default=[1,2,3],class_=int)

Overwriting model.py


In [5]:
import model
reload(model);

In [6]:
thing = model.Thing(a=2.5, b=True, c=[4,5,6])
thing

Thing(a=2.5, b=True, c=[4, 5, 6], name='Thing00002')

### yaml

In [7]:
serialize.to_yaml(thing,stream=open('thing.yaml','w'))

In [8]:
!cat thing.yaml

!model.Thing
a: 2.5
b: true
c: [4, 5, 6]


In [9]:
serialize.from_yaml(open('thing.yaml'))

Thing(a=2.5, b=True, c=[4, 5, 6], name='Thing00003')

### json

In [10]:
serialize.to_json(thing,stream=open('thing.json','w'))

In [11]:
!cat thing.json

{
    "py/obj": {
        "kwargs": {
            "a": 2.5,
            "b": true,
            "c": [
                4,
                5,
                6
            ]
        },
        "name": "model.Thing"
    }
}

In [12]:
serialize.from_json(open('thing.json'))

Thing(a=2.5, b=True, c=[4, 5, 6], name='Thing00004')

Any point having a "simple" json format that works for parameters only, supporting only json literals?

```
{
    "a": 2.5,
    "b": true,
    "c": [4,5,6]
}

which could be explicitly passed to model.Thing to recreate object. Or could include the name:


{
    "__name": "model.Thing",
         "a": 2.5,
         "b": true,
         "c": [4,5,6]
}
```


### script_repr

In [13]:
thing_ = thing.script_repr()
print(thing_)

model.Thing(a=2.5,
    b=True,
    c=[4, 5, 6])


In [14]:
eval(thing_)

Thing(a=2.5, b=True, c=[4, 5, 6], name='Thing00005')

## Attempt to load invalid parameter

In [15]:
%%writefile test2.yaml

!model.Thing
a: 10
b: false
c: [1, 2, 3]

Overwriting test2.yaml


In [16]:
try:
    serialize.from_yaml(open('test2.yaml'))
except ValueError as e:
    print(e.args[0])
else:
    raise AssertionError

Parameter 'a' must be at most 3


## More complex example

In [17]:
%%writefile model2.py

import param
import numpy
import collections

def fn(a,b):
    return a*b

class Thing(param.Parameterized):
    thing_param = param.Parameter(default=1)

class SomeThing(Thing):
    something_number = param.Number(default=2)
    something_array = param.Array(default=numpy.array([2,2,2]))
    something_array2 = param.Array(default=numpy.array([2,2,2]))
    something_list = param.List(default=[2,2])
    something_bool = param.Boolean(default=False)
    something_string = param.String(default='two')
    something_tupl = param.Tuple(default=(2,2,2,2))
    something_call = param.Callable(default=fn)

class Example(param.Parameterized):
    display_name = param.String()
    list_of_things = param.List()
    an_object = param.ClassSelector(Thing,default=SomeThing())
    conf_num = param.Number(default=None,bounds=(-0.5,0.5))
    conf_odict = param.Dict(default=collections.OrderedDict([(1,2),(3,4)]))
    conf_complex = param.Number(default=1+2j)
    conf_deque = param.Parameter(default=collections.deque('abc'))
    #list_of_pain = param.List([SomeThing(),SomeThing()])

Overwriting model2.py


In [18]:
import model2
reload(model2);

In [19]:
example = model2.Example()
example

Example(an_object=SomeThing(name='SomeThing00009', something_array=array([2, 2, 2]), something_array2=array([2, 2, 2]), something_bool=False, something_call=<function fn at 0x113f34bf8>, something_list=[2, 2], something_number=2, something_string='two', something_tupl=(2, 2, 2, 2), thing_param=1), conf_complex=(1+2j), conf_deque=deque(['a', 'b', 'c']), conf_num=None, conf_odict=OrderedDict([(1, 2), (3, 4)]), display_name='', list_of_things=[], name='Example00008')

### yaml

In [20]:
serialize.to_yaml(example,stream=open('example.yaml','w'))

In [21]:
!cat example.yaml

!model2.Example
an_object: !model2.SomeThing
  something_array: !numpy.array
  - [2, 2, 2]
  something_array2: !numpy.array
  - [2, 2, 2]
  something_bool: false
  something_call: !model2.fn
  something_list: [2, 2]
  something_number: 2
  something_string: two
  something_tupl: !tuple [[2, 2, 2, 2]]
  thing_param: 1
conf_complex: !builtins.complex [1.0, 2.0]
conf_deque: !collections.deque
- [a, b, c]
conf_num: null
conf_odict: !collections.OrderedDict
- - !tuple [[1, 2]]
  - !tuple [[3, 4]]
display_name: ''
list_of_things: []


In [22]:
example_ = serialize.from_yaml(open('example.yaml'))
example_

Example(an_object=SomeThing(name='SomeThing00010', something_array=array([2, 2, 2]), something_array2=array([2, 2, 2]), something_bool=False, something_call=<function fn at 0x113f34bf8>, something_list=[2, 2], something_number=2, something_string='two', something_tupl=(2, 2, 2, 2), thing_param=1), conf_complex=(1+2j), conf_deque=deque(['a', 'b', 'c']), conf_num=None, conf_odict=OrderedDict([(1, 2), (3, 4)]), display_name='', list_of_things=[], name='Example00011')

### json

In [23]:
serialize.to_json(example,stream=open('example.json','w'))

In [24]:
!cat example.json

{
    "py/obj": {
        "kwargs": {
            "an_object": {
                "py/obj": {
                    "kwargs": {
                        "something_array": {
                            "py/obj": {
                                "args": [
                                    [
                                        2,
                                        2,
                                        2
                                    ]
                                ],
                                "name": "numpy.array"
                            }
                        },
                        "something_array2": {
                            "py/obj": {
                                "args": [
                                    [
                                        2,
                                        2,
                                        2
                                    ]
                                ],
                               

In [29]:
!pip install jsonpickle

In [30]:
import jsonpickle
jsonpickle.dumps(example)

'{"py/object": "model2.Example", "py/state": {"_an_object_param_value": {"py/object": "model2.SomeThing", "py/state": {"_name_param_value": "SomeThing00009", "_something_array2_param_value": {"py/reduce": [{"py/object": "__builtin__.builtin_function_or_method"}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg==\\n"}]}, {"py/tuple": [1, {"py/tuple": [3]}, {"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["i8", 0, 1]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}, null, null]}, false, {"py/b64": "AgAAAAAAAAACAAAAAAAAAAIAAAAAAAAA\\n"}]}, null, null]}, "_something_array_param_value": {"py/reduce": [{"py/id": 3}, {"py/tuple": [{"py/type": "numpy.ndarray"}, {"py/tuple": [0]}, {"py/b64": "Yg==\\n"}]}, {"py/tuple": [1, {"py/tuple": [3]}, {"py/id": 4}, false, {"py/b64": "AgAAAAAAAAACAAAAAAAAAAIAAAAAAAAA\\n"}]}, null, null]}, "_something_list_param_value": [2, 2], "initialized": true}}, "_conf_odict_param_value": {"py/reduce": [{"py/type": "collections

### script_repr

## Notes

* same underlying mechanism used for to_yaml/from_yaml, script_repr/eval, and to_json/from_json